In [115]:
from pathlib import Path

import numpy as np
import pandas as pd
from numba import njit, prange

from matplotlib import pyplot as plt

In [116]:

def F(x: np.ndarray) -> float:
    return 1/(1+np.exp(-x))


def dF(x: np.ndarray) -> float:
    return F(x)*(1-F(x))


@njit(fastmath=True)
def F_jit(x: np.ndarray) -> float | np.ndarray:
    return 1/(1+np.exp(-x))


@njit(fastmath=True)
def dF_jit(x: np.ndarray) -> float | np.ndarray:
    return F_jit(x)*(1-F_jit(x))


@njit(fastmath=True)
def grads(xBatch: np.ndarray, yBatch: np.ndarray, w: np.ndarray, b: np.ndarray) -> tuple[np.ndarray]:
    n = xBatch.shape[0]
    nOut = w.shape[1]

    dw = np.zeros(w.shape)
    db = np.zeros(b.shape)
    
    for i in prange(n):
        u = np.dot(xBatch[i], w)+b
        a = F_jit(u)

        dEdu = 2/nOut*(a-yBatch[i])*dF_jit(u)
        dw += dEdu*np.atleast_2d(xBatch[i]).T
        db += dEdu
    
    return dw, db

class NNClassifier:
    def __init__(self, nIn: int, nOut: int) -> None:
        self.nIn = nIn
        self.nOut = nOut
        self.w: np.ndarray = np.random.uniform(-1, 1, (nIn, nOut))
        self.b: np.ndarray = np.zeros((nOut))


    def predict(self, x:np.ndarray) -> np.ndarray:
        return F(np.dot(x, self.w)+self.b)


    def train(self, xTrain: np.ndarray, yTrain: np.ndarray, lr, batch_size, max_iter) -> None:
        n = xTrain.shape[0]

        for k in range(max_iter):
            idxs = np.random.choice(a=np.arange(n), size=batch_size, replace=False)

            dw, db = grads(xTrain[idxs], yTrain[idxs], self.w, self.b)
            
            self.w -= lr*dw
            self.b -= lr*db
        
    
    def loss(self, x: np.ndarray, y: np.ndarray) -> float:
        n = y.shape[0]
        
        d = np.array([1/self.nOut*np.sum(np.square(self.predict(xi)-yi)) for xi, yi in zip(x, y)])
        
        return 1/n*np.sum(d)

In [117]:
df = pd.read_csv(Path('..', '..', 'data', 'iris_csv.csv'))

for c in df.columns[0:4]:
    df[c] = (df[c]-df[c].min())/(df[c].max()-df[c].min())

df['synth1'] = df['petallength']*df['petalwidth']
df['synth2'] = df['sepallength']*df['petallength']
df['synth3'] = df['sepallength']*df['petalwidth']

for name in df['class'].unique():
    df[f'{name}_label'] = df['class'].map(lambda x: 1 if x == name else 0)

In [123]:
test_frac = 0.8

p = np.random.permutation(df.index.size)

test_size = int(p.size*test_frac)
train_size = int(p.size*(1-test_frac))

idx_test = p[0 : test_size]
idx_train = p[test_size: p.size]

features_columns = ['sepallength', 'sepalwidth', 'synth1']
label_columns = ['Iris-setosa_label', 'Iris-versicolor_label', 'Iris-virginica_label']

xTest = np.array(df.iloc[idx_test][features_columns])
yTest = np.array(df.iloc[idx_test][label_columns])

xTrain = np.array(df.iloc[idx_train][features_columns])
yTrain = np.array(df.iloc[idx_train][label_columns])

In [125]:
nIn = 3
nOut = 3

learning_rate = 1e-1
batch_size = 10
max_iter = 15000

model = NNClassifier(nIn, nOut)

print('untrained loss: ', model.loss(xTest, yTest).round(4))

model.train(
    xTrain,
    yTrain,
    learning_rate,
    batch_size,
    max_iter
)

print('trained loss: ', model.loss(xTest, yTest).round(4))

untrained loss:  0.2761
trained loss:  0.057


In [126]:
print(model.w.round(3))
print(model.b.round(3))

[[ -6.933   9.135  -2.178]
 [  9.439 -12.093  -3.071]
 [ -8.296 -11.285  17.356]]
[-1.087  3.777 -4.947]
